In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import json
from PIL import Image
from io import BytesIO
import os
import re

def fetch_text(string):
    s =string.replace('..','')

    m=re.split(r'([,.])', s)
    conter=0
    a=[]
    for n in m:
        if conter+1==len(m):
            if m[len(m)-1]!='.' and m[len(m)-1]!=',' and m[len(m)-1]!='':
                a.append(m[len(m)-1])
            break
        
        a.append(m[conter]+m[conter+1])
        conter+=2

    return a


def fetch_screenshots(submission, dump_dir="dump", night_mode=False, limit=3):
    """
    Takes screenshots of all top level comments for a given reddit post.
    Dumps them into the specified dump_dir along with the text each omment has.
    """
#     _browser_profile = webdriver.FirefoxProfile()
#     _browser_profile.set_preference("dom.webnotifications.enabled", False)
#     driver = webdriver.Firefox(executable_path = '/Users/rahul/Desktop/test1/geckodriver',firefox_profile=_browser_profile)
#     #driver = webdriver.Firefox(firefox_profile=_browser_profile)
    
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    driver = webdriver.Chrome(executable_path = '/Users/rahul/Desktop/test1/chromedriver', chrome_options=chrome_options)

    print("Loading page...")
    driver.get("https://www.reddit.com/login/")
    driver.find_element_by_id('loginUsername').send_keys('')
    driver.find_element_by_id('loginPassword').send_keys('')
    driver.find_element_by_class_name('AnimatedForm__submitButton').click()
    
    driver.get(submission)
    driver.maximize_window()
    

#     if submission.over_18:
#         yes = driver.find_element_by_class_name("bzs6dt-10")
#         yes.click()
########################
#     Enable night mode on reddit

    ########################

    try:
        view_entire_discussion = driver.find_element_by_class_name(
            "j9NixHqtN2j8SKHcdJ0om"
        )
        view_entire_discussion.click()
    except:
        pass

    # LOAD PAGE FULLY
    enabel_dark=0
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(0.3)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    driver.execute_script("window.scrollTo(0, 0);")
    
    
    #print(driver.find_element_by_class_name("Post"))
        # Enable night mode on reddit
        
    if night_mode and  enabel_dark==0:
            print("Enabling night mode")
            dropdown = driver.find_element_by_id("USER_DROPDOWN_ID")
            dropdown.click()
            try:
                driver.execute_script("""document.querySelectorAll('button[aria-checked="false"]')[0].click();""") 
                
            except:
                pass
            
            dropdown.click()  # hide again
            time.sleep(1)
    driver.execute_script("document.getElementsByTagName('header')[0].remove();")
    time.sleep(1)
    #driver.execute_script("document.getElementsByClassName('K4Eem3pMbRbAYioOfqN5E')[0].remove();")
    #time.sleep(1)
    driver.execute_script("document.getElementsByClassName('_3obgdFz00GvpqpuX8QCsNK')[0].remove();")
    time.sleep(1)
    driver.execute_script("arguments[0].scrollIntoView();", driver.find_element_by_class_name("Post"))
    time.sleep(1)
    ttt=driver.find_element_by_class_name("Post")
    ppp=ttt.find_elements_by_tag_name('h1')
    ppp2=ttt.find_elements_by_tag_name('p')
    title_text={}
    fin1=[]
    fin2=[]
    for pp in ppp:
        fin1.append(pp.text)
    for pp2 in ppp2:
        fin2.append(pp2.text)
    for pp in ppp:
        driver.execute_script('arguments[0].innerHTML = "";', pp)
    for pp2 in ppp2:
        driver.execute_script('arguments[0].innerHTML = "";', pp2)    

    counter=0
    counter22=0
        
    for t in ppp:
            m=fetch_text(fin1[counter])
            counter=counter+1
            for n in m:
                if n == "" or not n :
                    continue
                driver.execute_script('arguments[0].innerHTML += arguments[1];', t,n)
                #time.sleep(1)
                if counter22==0:
                    n='r slash ask read it... '+n
                title_text[counter22] =n        
                ttt.screenshot(f"{dump_dir}/{counter22}.png")
                print("Took screenshot of title  number ", counter22)
                counter22=counter22+1

    counter=0
    for t in ppp2:
            m=fetch_text(fin2[counter])
            counter=counter+1
            for n in m:
                if n == "" or not n :
                    continue
                driver.execute_script('arguments[0].innerHTML += arguments[1];', t,n)

                title_text[counter22] =n        
                ttt.screenshot(f"{dump_dir}/{counter22}.png")
                #time.sleep(99999)
                print("Took screenshot of title  number ", counter22)
                counter22=counter22+1               

    title_count=counter22
    counter22=0

        
    driver.get(submission+'?sort=top')
    
    
    
    #comments = driver.find_elements_by_class_name("top-level")

    try:
        view_entire_discussion = driver.find_element_by_class_name(
            "j9NixHqtN2j8SKHcdJ0om"
        )
        view_entire_discussion.click()
    except:
        pass

    # LOAD PAGE FULLY

    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(0.3)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    driver.execute_script("window.scrollTo(0, 0);")        
        
        
        
    comment_text = {}
    comment_counter={}
    counter3=0
    
   
    counter22=1
    # now that we have the preliminary stuff out of the way time to get that image :D
    mmm = driver.find_element_by_class_name('_1YCqQVO-9r-Up6QPB9H6_4') # find part of the page you want image of
    element=mmm.find_elements_by_xpath("./*")
    c=0
    elements=[]
    morecomments=[]
    for e in element:
        
        morecomments=e.find_elements_by_class_name('_2HYsucNpMdUpYlGBMviq8M')
        if len(morecomments) ==0:
            test=e.find_elements_by_class_name('_36AIN2ppxy_z-XSDxTvYj5')
            if len(test) ==1:
                elements.append(e.find_element_by_class_name('_1sA-1jNHouHDpgCp1fCQ_F').get_attribute("href"))
                
        
    for e in elements:
        if c==limit:
            break
        c+=1
    
        driver.get(e)

        # Enable night mode on reddit
        
        if night_mode and  enabel_dark==0:
            print("Enabling night mode")
            dropdown = driver.find_element_by_id("USER_DROPDOWN_ID")
            dropdown.click()
            try:
                driver.execute_script("""document.querySelectorAll('button[aria-checked="false"]')[0].click();""") 
                
            except:
                pass
            
            dropdown.click()  # hide again
            enabel_dark=1
            time.sleep(1)

        try:
            view_entire_discussion = driver.find_element_by_class_name(
                "j9NixHqtN2j8SKHcdJ0om"
            )
            view_entire_discussion.click()
        except:
            pass
        try:
            view_entire_discussion = driver.find_element_by_class_name(
                "j9NixHqtN2j8SKHcdJ0om"
            )
            view_entire_discussion.click()
        except:
            pass

        mmm = driver.find_element_by_class_name('_1YCqQVO-9r-Up6QPB9H6_4') # find part of the page you want image of
        comment=mmm.find_elements_by_xpath("./*")
        comments=[]
       
        mn=0
        for nnn in comment :
            
            if len(nnn.find_elements_by_class_name('icon-upvote'))>0:
                comments.append(nnn)
                mn+=1
                if mn==2:
                    break
        counter2=0 
        ###############
        driver.execute_script("document.getElementsByTagName('header')[0].remove();")
        time.sleep(1)
        driver.execute_script("arguments[0].scrollIntoView();", comments[0])
        time.sleep(1)
        text = comments[0].find_elements_by_tag_name('p')
        upvot=comments[0].find_element_by_xpath('//button[@aria-label="upvote"]').get_attribute('aria-pressed')
        print(upvot)
        if upvot=="false":
            comments[0].find_element_by_class_name('icon-upvote').click()
        fin=[]

        for t in text:
            fin.append(t.text)

        for t in text:
            driver.execute_script('arguments[0].innerHTML = "";', t)
            #time.sleep(1)
        counter=0
        
        for t in text:
            m=fetch_text(fin[counter])
            counter=counter+1
            for n in m:
                if n == "" or not n :
                    continue
                driver.execute_script('arguments[0].innerHTML += arguments[1];', t,n)
                #time.sleep(1)
                comment_text[counter22] =n        
                comments[0].screenshot(f"{dump_dir}/comment-{counter22}.png")
                print("Took screenshot of comment", counter22)
                counter22=counter22+1
                counter2=counter2+1

                ###############################
               
        try:
            driver.execute_script("arguments[0].scrollIntoView();", comments[1])
            time.sleep(1)
            im = Image.open(f"{dump_dir}/comment-{counter22-1}.png")
            im1=im

            text = comments[1].find_elements_by_tag_name('p')
            upvot=comments[1].find_element_by_xpath('//button[@aria-label="upvote"]').get_attribute('aria-pressed')
            print(upvot)
            if upvot=="false":
                comments[1].find_element_by_class_name('icon-upvote').click()
            fin=[]

            for t in text:
                fin.append(t.text)

            for t in text:
                driver.execute_script('arguments[0].innerHTML = "";', t)
                #time.sleep(1)
            counter=0
            
            for t in text:
                m=fetch_text(fin[counter])
                counter=counter+1
                for n in m:
                    if n == "" or not n :
                        continue
                    driver.execute_script('arguments[0].innerHTML += arguments[1];', t,n)
                    #time.sleep(1)
                    comment_text[counter22] =n 



                    comments[1].screenshot('test.png')
                    im = Image.open('test.png') # uses PIL library to open image in memory
                    images_list = [im1, im]
                    imgs=[im1,im]

                    # If you're using an older version of Pillow, you might have to use .size[0] instead of .width
                    # and later on, .size[1] instead of .height
                    min_img_width = min(i.width for i in imgs)

                    total_height = 0
                    for i, img in enumerate(imgs):
                        # If the image is larger than the minimum width, resize it
                        if img.width > min_img_width:
                            imgs[i] = img.resize((min_img_width, int(img.height / img.width * min_img_width)), Image.ANTIALIAS)
                        total_height += imgs[i].height

                    # I have picked the mode of the first image to be generic. You may have other ideas
                    # Now that we know the total height of all of the resized images, we know the height of our final image
                    img_merge = Image.new(imgs[0].mode, (min_img_width, total_height))
                    y = 0
                    for img in imgs:
                        img_merge.paste(img, (0, y))

                        y += img.height
                    img_merge.save(f"{dump_dir}/comment-{counter22}.png")
                    os.remove("test.png")


                    print("Took screenshot of comment", counter22)
                    counter22=counter22+1
                    counter2=counter2+1
        except:
            jjjjj=0



    #driver.quit()

        comment_counter[counter3]=counter2
        counter3=counter3+1



    #driver.quit()
    print(comment_counter)
    return comment_text,comment_counter,counter22-1,title_text,title_count


In [ ]:
# Audio!
import pyttsx3
import json
import time


def generate_audio(title_text, title_count,comment_text, end, start):
    
    engine = pyttsx3.init()

    engine.setProperty("rate", 190)
    voices = engine.getProperty("voices")

    try:
        voice = [v for v in voices if v.name == "Daniel"][0]
    except IndexError:
        pass
    else:
        engine.setProperty("voice", voice.id)

    for i in range (0,title_count):
            
        engine.save_to_file(title_text[i], f"dump/{i}.mp3")
        
    engine.save_to_file(start, "dump/start.mp3")
    engine.save_to_file(end, "dump/end.mp3")

    for id, text in comment_text.items():
        if(len(text) == 1 and text == "."):
            text = "following"
            engine.save_to_file(text, f"dump/audio-{id}.mp3")
        elif(len(text) == 1 and text == "]"):
            text = "next"
            engine.save_to_file(text, f"dump/audio-{id}.mp3")
        
            
            
        engine.save_to_file(text, f"dump/audio-{id}.mp3")
        time.sleep(0.1)
        print("Generating audio for comment", id)

    engine.runAndWait()

In [ ]:
# Video!
import os
import shutil
from moviepy.editor import *
def composite_video(comment_counter,counter22,title_count):

    moviesize = (1280, 720)

    #_dir = sorted(os.listdir("dump"))
    #audio_clips = [AudioFileClip(f"dump/{x}") for x in _dir if x.startswith("audio")]
    #image_clips = [ImageClip(f"dump/{x}") for x in _dir if x.startswith("comment")]
    audio_clips=[]
    image_clips=[]
    title_audio_clips=[]
    title_image_clips=[]    
    for i in  range (1,counter22+1):
        audio_clips.append(AudioFileClip(f"dump/audio-{i}.mp3"))
        image_clips.append(ImageClip(f"dump/comment-{i}.png"))
    background = ColorClip(moviesize, color=(26, 26, 27))

    intermission = (
        VideoFileClip("data/transition.mp4").set_duration(0.7).resize(height=720)
    )


    for i in  range (0,title_count):
        title_audio_clips.append(AudioFileClip(f"dump/{i}.mp3"))
        title_image_clips.append(ImageClip(f"dump/{i}.png"))
        
    final =[]
    start_audio = AudioFileClip("dump/start.mp3")
    for audio, image in zip(title_audio_clips, title_image_clips):
        
        title = (
        CompositeVideoClip([background, image.set_pos("center").resize(0.7)], size=moviesize)
        .set_audio(audio)
        .set_duration(audio.duration))
        
        final.append(title)

    title = (
        CompositeVideoClip([background, image.set_pos("center").resize(0.7)], size=moviesize)
        .set_audio(start_audio)
        .set_duration(start_audio.duration))
    final.append(title)


    
    #
#     start_audio = AudioFileClip("dump/start.mp3")
#     final.append(background.set_duration(start_audio.duration).set_audio(start_audio))
    #
    final.append(intermission)
    
    counter=0
    counter2=0
    for audio, image in zip(audio_clips, image_clips):
        
        clip = (
            CompositeVideoClip(
                [background, image.set_pos("center").resize(0.60)], size=moviesize
            )
            .set_duration(audio.duration)
            .set_audio(audio)
        )
        final.append(clip)
        counter2=counter2+1
        if comment_counter[counter]!=counter2:
            #print(counter2)
            
            continue
        counter=counter+1
        counter2=0
        final.append(intermission)

    end_audio = AudioFileClip("dump/end.mp3")
    final.append(background.set_duration(end_audio.duration).set_audio(end_audio))

    mins, secs = divmod(sum(x.duration for x in final), 60)
    print(f"Duration of video: {int(mins)}m, {int(secs)}s")

    vid = concatenate_videoclips(final)

    bg_music = afx.audio_loop(
        AudioFileClip("data/light-music.mp3"), duration=vid.duration
    ).volumex(0.6)
    final_audio = CompositeAudioClip([vid.audio, bg_music])
    vid = vid.set_audio(final_audio)
    vid.write_videofile("video.mp4", fps=20, codec="libx264", audio_codec="aac")

    shutil.rmtree("dump")



In [ ]:

# Running APP
import os
import sys
import time
import json

import praw


if "dump" not in os.listdir():
    os.mkdir("dump")

reddit = praw.Reddit()

if __name__ == "__main__":
    
    submission = input("enter the url : ")
    
    comment_text,comment_counter,counter22,title_text,title_count = fetch_screenshots(submission,night_mode = True, limit = 3)
    print(comment_counter)
    
    #title_text = "r slash ask read it... " + title  # lol
    generate_audio(
        
    )
    composite_video(comment_counter,counter22,title_count)




In [ ]:
# https://www.reddit.com/r/AskReddit/comments/cyffwp/what_is_your_default_thought_when_you_are_bored/